In [54]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики

from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import preprocessing #предобработка

from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [55]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [56]:
data.shape

(3751, 1777)

##### Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
##### Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов
##### Предварительная обработка не требуется, данные уже закодированы и нормализованы.
##### В качестве метрики будем использовать F1-score.
##### Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [57]:
#Создаем матрицу наблюдений X и столбец с ответами y
X = data.drop('Activity', axis=1)
Y = data['Activity']

In [58]:
#Создаем тренировочную и тестовую выборки с помощью train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, #таблица с наблюдениями и столбец с правильным ответами
    test_size=0.2, #размер тестовой выборки
    random_state=40 #число отвечающее за генерацию случайных чисел
)
#Выводим размеры полученных выборок
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (3000, 1776)
Test shape: (751, 1776)


##### Обучение модели Логистическая регрессия

In [59]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=500)

log_reg.fit(X_train, Y_train) #Обучаем модель
Y_lr_pred = log_reg.predict(X_test) #Делаем предсказание на тестовой выборке
print(metrics.classification_report(Y_test, Y_lr_pred))
print('F1_score Логистической регрессии: {:.2f}'.format(metrics.f1_score(Y_test, Y_lr_pred)))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       359
           1       0.74      0.78      0.76       392

    accuracy                           0.75       751
   macro avg       0.75      0.74      0.74       751
weighted avg       0.75      0.75      0.75       751

F1_score Логистической регрессии: 0.76


##### Обучение модели Случайный лес

In [60]:
rand_clf = ensemble.RandomForestClassifier(random_state=42) #модель RandomForest без параметров (только random_state для воспроизводимости)
rand_clf.fit(X_train, Y_train)
Y_rf_pred = rand_clf.predict(X_test)
#Выводим отчет о метриках
print(metrics.classification_report(Y_test, Y_rf_pred))
print('F1_score модели Случайный лес: {:.2f}'.format(metrics.f1_score(Y_test, Y_rf_pred)))

              precision    recall  f1-score   support

           0       0.81      0.78      0.80       359
           1       0.81      0.83      0.82       392

    accuracy                           0.81       751
   macro avg       0.81      0.81      0.81       751
weighted avg       0.81      0.81      0.81       751

F1_score модели Случайный лес: 0.82


##### Оптимизация параметров модели Логистической регрессии с помощью Grid Search

In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
parametrs1= {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }

grid1 = GridSearchCV (log_reg, parametrs1, cv=5 )  #запускаем обучение
grid1.fit(X_train,Y_train)

c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\olgav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs fa

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=500, random_state=42),
             param_grid={'penalty': ['l2', 'none'],
                         'solver': ['lbfgs', 'saga']})

In [63]:
grid1.best_params_  # выводим результат

{'penalty': 'l2', 'solver': 'saga'}

In [64]:
print("Наилучшая модель:\n{}".format(grid1.best_estimator_))

Наилучшая модель:
LogisticRegression(max_iter=500, random_state=42, solver='saga')


In [65]:
log_reg1 = linear_model.LogisticRegression(random_state=42, max_iter=1000)
log_reg1.fit(X_train, Y_train) #Обучаем модель
Y_lr_pred1 = log_reg1.predict(X_test) #Делаем предсказание на тестовой выборке
print(metrics.classification_report(Y_test, Y_lr_pred1))
print('F1_score Логистической регрессии: {:.2f}'.format(metrics.f1_score(Y_test, Y_lr_pred1)))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       359
           1       0.74      0.78      0.76       392

    accuracy                           0.75       751
   macro avg       0.75      0.74      0.74       751
weighted avg       0.75      0.75      0.75       751

F1_score Логистической регрессии: 0.76


##### F1 = 0.76 после Grid Search оптимизации параметров во модели логистической регрессии. В модели с параметрами по умолчанию F1 был равен 0.76 (результат такой же как и в модели по умолчанию)

##### Оптимизация параметров модели Случайный лес с помощью Grid Search

In [66]:
parametrs = { 'n_estimators': range (10, 51, 10), 
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

grid = GridSearchCV(rand_clf, parametrs, cv=5) #запускаем обучение
grid.fit(X_train,Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(10, 51, 10)})

In [67]:
grid.best_params_  # выводим результат


{'max_depth': 11,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [68]:
print("Наилучшая модель:\n{}".format(grid.best_estimator_))

Наилучшая модель:
RandomForestClassifier(max_depth=11, n_estimators=50, random_state=42)


In [69]:
rand_clf = ensemble.RandomForestClassifier(max_depth=11, n_estimators=50, random_state=42) #запускаем модель с оптимальными параметрами
rand_clf.fit(X_train, Y_train)
Y_rf_pred = rand_clf.predict(X_test)
#Выводим отчет о метриках
print(metrics.classification_report(Y_test, Y_rf_pred))
print('F1_score модели Случайный лес: {:.2f}'.format(metrics.f1_score(Y_test, Y_rf_pred)))

              precision    recall  f1-score   support

           0       0.80      0.75      0.77       359
           1       0.78      0.82      0.80       392

    accuracy                           0.79       751
   macro avg       0.79      0.79      0.79       751
weighted avg       0.79      0.79      0.79       751

F1_score модели Случайный лес: 0.80


##### F1 = 0.8 после Grid Search оптимизации параметров в модели Случайный лес. В модели с параметрами по умолчанию F1 был равен 0.82 (странный результат)

#### Оптимизация гиперпараметров логистической регрессии с помощью RandomsizedSearchCV

In [70]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=1000)

log_reg.fit(X_train, Y_train) #Обучаем модель
Y_lr_pred = log_reg.predict(X_test) #Делаем предсказание на тестовой выборке
print(metrics.classification_report(Y_test, Y_lr_pred))
print('F1_score Логистической регрессии: {:.2f}'.format(metrics.f1_score(Y_test, Y_lr_pred)))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       359
           1       0.74      0.78      0.76       392

    accuracy                           0.75       751
   macro avg       0.75      0.74      0.74       751
weighted avg       0.75      0.75      0.75       751

F1_score Логистической регрессии: 0.76


In [71]:
from sklearn.model_selection import RandomizedSearchCV
param_lr_2 = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_lr_2, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1 )  
%time random_search.fit(X_train, Y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, Y_test)))
Y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(Y_test, Y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 2.25 s
Wall time: 3min 20s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.45}


In [72]:
print("Наилучшая модель:\n{}".format(random_search.best_estimator_))

Наилучшая модель:
LogisticRegression(C=0.45, max_iter=1000, random_state=42)


In [73]:
log_reg2 = linear_model.LogisticRegression(C=0.56, max_iter=1000, random_state=42, solver='sag')

log_reg2.fit(X_train, Y_train) #Обучаем модель
Y_lr_pred2 = log_reg2.predict(X_test) #Делаем предсказание на тестовой выборке
print(metrics.classification_report(Y_test, Y_lr_pred2))
print('F1_score Логистической регрессии: {:.2f}'.format(metrics.f1_score(Y_test, Y_lr_pred2)))

              precision    recall  f1-score   support

           0       0.75      0.70      0.73       359
           1       0.74      0.79      0.76       392

    accuracy                           0.75       751
   macro avg       0.75      0.75      0.75       751
weighted avg       0.75      0.75      0.75       751

F1_score Логистической регрессии: 0.76


##### F1 = 0.76 после RandomSizedSearch оптимизации параметров во модели логистической регрессии. В модели с параметрами по умолчанию, а также после оптимизации Grid Search F1 был равен 0.76 (без изменений)

#### Оптимизация гиперпараметров модели Случайный лес с помощью RandomsizedSearchCV

In [74]:
rand_clf = ensemble.RandomForestClassifier(max_depth=11, n_estimators=50, random_state=42) # модель после потимизации Grid Search
rand_clf.fit(X_train, Y_train)
Y_rf_pred = rand_clf.predict(X_test)
print('F1_score модели Случайный лес: {:.2f}'.format(metrics.f1_score(Y_test, Y_rf_pred)))

F1_score модели Случайный лес: 0.80


In [75]:
param_rf_2 = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int)) }
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_rf_2, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
random_search_forest.fit(X_train, Y_train)
Y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(Y_test, Y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))  
print("Наилучшая модель:\n{}".format(random_search_forest.best_estimator_))    

f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 28}
Наилучшая модель:
RandomForestClassifier(max_depth=28, min_samples_leaf=5, n_estimators=170,
                       random_state=42)


##### F1_score = 0.81 в модели Случайный лес, он увеличился посли RandomSizedSearch оптимизации

##### Кросс-валидация KFold

### ML7 - Оптимизация гиперпараметров

In [76]:
from sklearn import model_selection #методы разделения и валидации
#наилучшая модель от RandomSizedSearchCV
model = ensemble.RandomForestClassifier(max_depth=20, min_samples_leaf=5, n_estimators=140,random_state=42)
#Создаём объект кросс-валидатора KFold
kf = model_selection.KFold(n_splits=5)

#Создаём список для хранения тренировочных и валидационных метрик
train_metrics = []
val_metrics = []
#Организуем цикл для кросс-валидации (используем весь набор данных)
#train_index — индексы тренировочной выборки
#valid_index — индексы валидационной выборки
for train_index, valid_index in kf.split(X, Y): 
    #Создаём тренировочную и валидационную выборку, обращаясь по текущим индексам
    X_train, Y_train = X.iloc[train_index], Y.iloc[train_index]
    X_valid, Y_valid = X.iloc[valid_index], Y.iloc[valid_index]
    #Обучаем случайный лес на тренировочной выборке
    model.fit(X_train, Y_train)
    #Делаем предсказание для каждой из выборок
    Y_train_pred = model.predict(X_train)
    Y_valid_pred = model.predict(X_valid)
    #Рассчитываем метрику и заносим её в список
    train_metrics.append(metrics.f1_score(Y_train, Y_train_pred))
    val_metrics.append(metrics.f1_score(Y_valid, Y_valid_pred))

In [77]:
print(train_metrics)
print(val_metrics)

[0.9395931142410014, 0.9396215079603484, 0.9410681399631676, 0.9448110377924415, 0.9453219927095992]
[0.8158508158508159, 0.8069306930693068, 0.7906976744186047, 0.8141592920353982, 0.8155339805825244]


In [78]:
print('Train k-fold mean F1: {:.2f}'.format(np.mean(train_metrics)))
print('Valid k-fold mean F1: {:.2f}'.format(np.mean(val_metrics)))

Train k-fold mean F1: 0.94
Valid k-fold mean F1: 0.81


##### Оптимизация параметров OPTUNA для модели Случайный лес

In [79]:
import optuna
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.1.0


In [80]:
random_state=42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, Y_train)
  score = metrics.f1_score(Y_train, model.predict(X_train))
  return score

In [81]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=15)

[I 2023-02-24 13:18:40,945] A new study created in memory with name: RandomForestClassifier
[I 2023-02-24 13:18:43,528] Trial 0 finished with value: 0.9124198962465669 and parameters: {'n_estimators': 188, 'max_depth': 13, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9124198962465669.
[I 2023-02-24 13:18:45,586] Trial 1 finished with value: 0.8939532057125493 and parameters: {'n_estimators': 166, 'max_depth': 12, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.9124198962465669.
[I 2023-02-24 13:18:47,784] Trial 2 finished with value: 0.9262195121951219 and parameters: {'n_estimators': 150, 'max_depth': 14, 'min_samples_leaf': 6}. Best is trial 2 with value: 0.9262195121951219.
[I 2023-02-24 13:18:50,921] Trial 3 finished with value: 0.9801768831960964 and parameters: {'n_estimators': 187, 'max_depth': 23, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.9801768831960964.
[I 2023-02-24 13:18:52,726] Trial 4 finished with value: 0.9011522134627047 and parameters: {'n

CPU times: total: 33.9 s
Wall time: 36.3 s


In [82]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 172, 'max_depth': 30, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99


In [83]:
rand_clf_3 = ensemble.RandomForestClassifier(max_depth=24, n_estimators=192, min_samples_leaf=2, random_state=42) 
rand_clf_3.fit(X_train, Y_train)
Y_rf_pred_3 = rand_clf_3.predict(X_test)
#Выводим отчет о метриках
print(metrics.classification_report(Y_test, Y_rf_pred_3))
print('F1_score модели Случайный лес: {:.2f}'.format(metrics.f1_score(Y_test, Y_rf_pred_3)))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94       359
           1       0.93      0.95      0.94       392

    accuracy                           0.94       751
   macro avg       0.94      0.94      0.94       751
weighted avg       0.94      0.94      0.94       751

F1_score модели Случайный лес: 0.94


##### OPTUNA для логистической регрессии

In [84]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
 # multi_class='multinomial', #мультиклассовая классификация
  max_iter = trial.suggest_int('max_iter', 500, 10000, 500)
  random_state=42
  # создаем модель
  model = linear_model.LogisticRegression(max_iter=max_iter, random_state=random_state)
 # обучаем модель
  model.fit(X_train, Y_train)
  score = metrics.f1_score(Y_train, model.predict(X_train))
  return score

In [85]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticsRegressionClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=15)

[I 2023-02-24 13:19:20,777] A new study created in memory with name: LogisticsRegressionClassifier
[I 2023-02-24 13:19:23,037] Trial 0 finished with value: 0.8970321017565113 and parameters: {'max_iter': 5500}. Best is trial 0 with value: 0.8970321017565113.
[I 2023-02-24 13:19:25,283] Trial 1 finished with value: 0.8970321017565113 and parameters: {'max_iter': 5000}. Best is trial 0 with value: 0.8970321017565113.
[I 2023-02-24 13:19:27,516] Trial 2 finished with value: 0.8970321017565113 and parameters: {'max_iter': 6000}. Best is trial 0 with value: 0.8970321017565113.
[I 2023-02-24 13:19:29,776] Trial 3 finished with value: 0.8970321017565113 and parameters: {'max_iter': 500}. Best is trial 0 with value: 0.8970321017565113.
[I 2023-02-24 13:19:32,036] Trial 4 finished with value: 0.8970321017565113 and parameters: {'max_iter': 9000}. Best is trial 0 with value: 0.8970321017565113.
[I 2023-02-24 13:19:34,278] Trial 5 finished with value: 0.8970321017565113 and parameters: {'max_iter

CPU times: total: 32.5 s
Wall time: 34.2 s


In [86]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'max_iter': 5500}
f1_score на обучающем наборе: 0.90


##### Оптимизация параметров с помощью Hyperopt

In [92]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import numpy as np
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))


Версия Hyperopt : 0.2.7


In [93]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [95]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, Y=Y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = space
    
    
   # {'n_estimators': int(params['n_estimators']), 
   #           'max_depth': int(params['max_depth']), 
   #          'min_samples_leaf': int(params['min_samples_leaf'])
#          }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, Y)
    score = metrics.f1_score(Y, model.predict(X))

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [ ]:
# запускаем hyperopt
trials = Trials() # используется для логирования результатов
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=random_state # фиксируем для повторяемости результата          
         )
print("Наилучшие значения гиперпараметров {}".format(best))

In [ ]:
# начинаем подбор гиперпараметров
#%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=random_state
          #rstate=np.random.RandomState(random_state) # фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test_scaled, y_test)))
y_test_pred = model.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


In [ ]:
# если необходимо продолжить подбор, 
# то увеличиваем max_evals(должен быть строго больше, чем на предыдущих итерациях) 
# и используем старый trials
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=23, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

In [ ]:
# запускаем hyperopt
trials = Trials()
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, pipeline=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций 
          # (можно ещё указать и время поиска) 
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            rstate=np.random.RandomState(1),
          # progressbar
            show_progressbar=True
        )